In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter

In [2]:
def change2Median(dataset:pd.DataFrame()):
    l= dataset.mean() - dataset.std()*3
    h= dataset.mean() + dataset.std()*3
    med = dataset.median()
    for i in dataset.index:
        if dataset.loc[i] < l or dataset.loc[i] > h:
            dataset.loc[i] = med
    return dataset

In [3]:
data_path =  os.getcwd() + "/dataset/original_data"
save_img = data_path + "/pic_filter"
freq = 64

b, a = butter(5, 16/freq, btype='lowpass')

In [4]:
subjects = []

for dirts in os.listdir(data_path):
    if dirts != 'pic':
        subjects.append(dirts)

In [5]:
subjects = ['S01R02.txt', 'S02R01.txt', 'S02R02.txt', 'S03R02.txt', 'S03R03.txt', 'S07R01.txt', 'S07R02.txt']

In [6]:
for sub in subjects:
    file = data_path + '/' + sub

    subData = pd.read_csv(file, delimiter=" ", header=None)
    subData.drop(index = list(subData[subData[10] == 0].index),
                 inplace = True)

    len = subData.shape[0]
    fog_index = []
    for i in subData.index:
        if subData.loc[i, 10] == 2: # 取index和'Action'對應到的資料
            fog_index.append(i) # 將發生fog的index存入fog_index這個array

    start_indices = []
    for i in fog_index:
        if (subData.loc[i-1, 10] != subData.loc[i, 10]):
            start_indices.append(i) # 將發生fog之前的index存入start_indices這個array

    prefog = []
    for start in start_indices:
        prefog_start = [x for x in range(start-freq, start)] # prefog_start把FOG之前的windows標記起來
        prefog.append(prefog_start)

    prefog = [item for sublist in prefog for item in sublist]

    for i in prefog:
        if subData.loc[i, 10] == 2:
            print (i)
        subData.loc[i, 10] = 3

    subData[10] = subData[10] - 1

    subData[3] = change2Median(subData[3])

    fig, ax = plt.subplots(figsize=(25, 4))
    x = np.arange(0, len/freq, 1/freq)
    y = lfilter(b, a, subData[3].values)
    z = subData[10].values

    plt.style.use("ggplot")
    plt.plot(x, y, 'r')
    ax.pcolorfast(ax.set_xlim(0, len/freq), ax.get_ylim(),
                  z[np.newaxis], cmap='Set3', alpha=0.3)
    image = save_img + '/' + sub.split(".")[0] + '.png'
    plt.savefig(image)
    plt.close(fig)
    del len


22456


In [7]:
subjects = ['S01R01.txt', 'S03R01.txt', 'S04R01.txt', 'S05R01.txt', 'S05R02.txt', 'S06R01.txt', 'S06R02.txt', 'S08R01.txt', 'S09R01.txt', 'S10R01.txt']

In [8]:
# subjects = ['S10R01.txt']

In [9]:
for sub in subjects:
    file = data_path + '/' + sub

    subData = pd.read_csv(file, delimiter=" ", header=None)
    subData = subData.drop(index = list(subData[subData[10] == 0].index),
                 inplace = False)


    fog_index = []
    for i in subData.index:
        if subData.loc[i, 10] == 2: # 取index和'Action'對應到的資料
            fog_index.append(i) # 將發生fog的index存入fog_index這個array

    start_indices = []
    for i in fog_index:
        if (subData.loc[i-1, 10] != subData.loc[i, 10]):
            start_indices.append(i) # 將發生fog之前的index存入start_indices這個array

    prefog = []
    for start in start_indices:
        prefog_start = [x for x in range(start-freq, start)] # prefog_start把FOG之前的windows標記起來
        prefog.append(prefog_start)

    prefog = [item for sublist in prefog for item in sublist]

    for i in prefog:
        if subData.loc[i, 10] == 2:
            print (i)
        subData.loc[i, 10] = 3

    subData[10] = subData[10] - 1
    subData[3] = change2Median(subData[3])

    count = 0
    start = 0
    time = list(subData[0])
    for i in range(subData.shape[0]):
        if i == subData.shape[0]-1:
            len = i - start
            fig, ax = plt.subplots(figsize=(25, 4))
            x = np.arange(0, len/freq, 1/freq)
            y = subData[3].values[start:i]
            z = subData[10].values[start:i]

            plt.style.use("ggplot")
            plt.plot(x, y, 'r')
            ax.pcolorfast(ax.set_xlim(0, len/freq), ax.get_ylim(),
                          z[np.newaxis], cmap='Set3', alpha=0.3)
            image = save_img + '/' + sub.split(".")[0] + '_' + str(count) + '.png'
            plt.savefig(image)
            plt.close(fig)
            start = i+1
            count += 1
            del len
            break
        if time[i+1] -20 > time[i]:
            len = i - start

            fig, ax = plt.subplots(figsize=(25, 4))
            # x = np.arange(0, len/freq, 1/freq)
            # y = subData[3].values[start:i]
            # z = subData[10].values[start:i]

            x = np.arange(0, 24, 1/freq)
            y = lfilter(b, a, subData[3].values[start:(start+freq*24)])
            z = subData[10].values[start:(start+freq*24)]

            plt.style.use("ggplot")
            plt.plot(x, y, 'r')
            # ax.pcolorfast(ax.set_xlim(0, len/freq), ax.get_ylim(),
            #               z[np.newaxis], cmap='Set3', alpha=0.3)
            ax.pcolorfast(ax.set_xlim(0, 24), ax.get_ylim(),
                          z[np.newaxis], cmap='Set3', alpha=0.3)
            image = save_img + '/' + sub.split(".")[0] + '_' + str(count) + '.png'
            plt.savefig(image)
            plt.close(fig)
            start = i+1
            count += 1
            del len
    